라이브러리 임포트

In [39]:
import pandas
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
# 지도 시각화
import folium


# 한글 깨짐 방지
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

plt.rc('font', family = 'NanumBarunGothic') # 나눔 바른 고딕을 기본 글꼴로 설정
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


데이터 준비하기

In [10]:
df = pandas.read_csv('/content/도로교통공단_교통사고 정보.csv', encoding = 'euc-kr')
df.head(3)

,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,사고유형_대분류,사고유형_중분류,사고유형,가해자법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
0,2019,2019010100,야,화,1,0,0,0,0,부산,해운대구,차대사람,기타,기타,중앙선 침범,교차로,교차로내,승용차,보행자,1150507.0,1685987.0,129.152465,35.157884
1,2019,2019010103,야,화,1,0,0,0,0,경기,안산시,차대차,정면충돌,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차,943018.0,1921233.0,126.857149,37.288292
2,2019,2019010116,주,화,1,0,0,0,0,경북,청도군,차대차,측면충돌,측면충돌,안전운전 의무 불이행,교차로,교차로부근,승용차,사륜오토바이(ATV),1100414.0,1739893.0,128.609230,35.650108


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   발생년         3233 non-null   int64  
 1   발생년월일시      3233 non-null   int64  
 2   주야          3233 non-null   object 
 3   요일          3233 non-null   object 
 4   사망자수        3233 non-null   int64  
 5   부상자수        3233 non-null   int64  
 6   중상자수        3233 non-null   int64  
 7   경상자수        3233 non-null   int64  
 8   부상신고자수      3233 non-null   int64  
 9   발생지시도       3233 non-null   object 
 10  발생지시군구      3233 non-null   object 
 11  사고유형_대분류    3233 non-null   object 
 12  사고유형_중분류    3233 non-null   object 
 13  사고유형        3233 non-null   object 
 14  가해자법규위반     3233 non-null   object 
 15  도로형태_대분류    3233 non-null   object 
 16  도로형태        3233 non-null   object 
 17  가해자_당사자종별   3233 non-null   object 
 18  피해자_당사자종별   3233 non-null   object 
 19  발생위치X_UTMK  3233 non-null  

교통사고 시간 전처리

In [11]:
# Subter 문자열 자르기를 위해 데이터타입 변경
df = df.astype({'발생년월일시' : 'string'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   발생년         3233 non-null   int64  
 1   발생년월일시      3233 non-null   string 
 2   주야          3233 non-null   object 
 3   요일          3233 non-null   object 
 4   사망자수        3233 non-null   int64  
 5   부상자수        3233 non-null   int64  
 6   중상자수        3233 non-null   int64  
 7   경상자수        3233 non-null   int64  
 8   부상신고자수      3233 non-null   int64  
 9   발생지시도       3233 non-null   object 
 10  발생지시군구      3233 non-null   object 
 11  사고유형_대분류    3233 non-null   object 
 12  사고유형_중분류    3233 non-null   object 
 13  사고유형        3233 non-null   object 
 14  가해자법규위반     3233 non-null   object 
 15  도로형태_대분류    3233 non-null   object 
 16  도로형태        3233 non-null   object 
 17  가해자_당사자종별   3233 non-null   object 
 18  피해자_당사자종별   3233 non-null   object 
 19  발생위치X_UTMK  3233 non-null  

In [12]:
#2019010100 -> 00 시간으로 분리
df['발생시간'] = df['발생년월일시'].str[-2:]
df['발생시간']

0       00
1       03
2       16
3       19
4       21
        ..
3228    05
3229    08
3230    12
3231    12
3232    16
Name: 발생시간, Length: 3233, dtype: string

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   발생년         3233 non-null   int64  
 1   발생년월일시      3233 non-null   string 
 2   주야          3233 non-null   object 
 3   요일          3233 non-null   object 
 4   사망자수        3233 non-null   int64  
 5   부상자수        3233 non-null   int64  
 6   중상자수        3233 non-null   int64  
 7   경상자수        3233 non-null   int64  
 8   부상신고자수      3233 non-null   int64  
 9   발생지시도       3233 non-null   object 
 10  발생지시군구      3233 non-null   object 
 11  사고유형_대분류    3233 non-null   object 
 12  사고유형_중분류    3233 non-null   object 
 13  사고유형        3233 non-null   object 
 14  가해자법규위반     3233 non-null   object 
 15  도로형태_대분류    3233 non-null   object 
 16  도로형태        3233 non-null   object 
 17  가해자_당사자종별   3233 non-null   object 
 18  피해자_당사자종별   3233 non-null   object 
 19  발생위치X_UTMK  3233 non-null  

In [15]:
# 발생시간 숫자 데이터타입으로 변경
df = df.astype({'발생시간' : 'int64'})

In [19]:
# 2019010100 -> 20190101 추출 -> 2019-01-01 (날짜형태 YYYY-MM-DD 형태로 변경)
df['발생년월일시'] = pandas.to_datetime(df['발생년월일시'].str[:8],
                                     format = '%Y-%m-%d', errors = 'raise')

In [21]:
df.head()

,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,사고유형_대분류,사고유형_중분류,사고유형,가해자법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X_UTMK,발생위치Y_UTMK,경도,위도,발생시간
0,2019,2019-01-01,야,화,1,0,0,0,0,부산,해운대구,차대사람,기타,기타,중앙선 침범,교차로,교차로내,승용차,보행자,1150507.0,1685987.0,129.152465,35.157884,0
1,2019,2019-01-01,야,화,1,0,0,0,0,경기,안산시,차대차,정면충돌,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차,943018.0,1921233.0,126.857149,37.288292,3
2,2019,2019-01-01,주,화,1,0,0,0,0,경북,청도군,차대차,측면충돌,측면충돌,안전운전 의무 불이행,교차로,교차로부근,승용차,사륜오토바이(ATV),1100414.0,1739893.0,128.609230,35.650108,16
3,2019,2019-01-01,야,화,1,0,0,0,0,대전,서구,차대사람,횡단중,횡단중,안전운전 의무 불이행,교차로,교차로부근,승용차,보행자,989194.0,1817621.0,127.379560,36.355946,19
4,2019,2019-01-01,야,화,1,0,0,0,0,경북,영천시,차대사람,기타,기타,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,1127700.0,1767912.0,128.915041,35.899506,21


2019년 지역별 교통사고 데이터 시각화

In [23]:
fig = px.scatter(df, x='발생년월일시', y = '발생시간', color = '발생지시도', hover_data = ['발생시간'])
fig.show()

In [26]:
fig = px.scatter(df, x='발생년월일시', y = '발생시간', color = '발생지시도', size = '사망자수', hover_data = ['발생시간'])
fig.show()

시간대 별 교통사고 사망자 현황

In [28]:
fig = px.bar(df, x = '사망자수', y = '발생시간', orientation = 'h')
fig.show()

시도별 교통사고 사망자 현황

In [30]:
fig = px.bar(df, x = '사망자수', y = '발생지시도', orientation = 'h', title = '시도별 교통사고 사망자 현황')
fig.show()

대구광역시의 교통사고 발생 데이터 분석

데이터 준비하기

In [32]:
df1 = df[df.발생지시도 == '대구']
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 21 to 3220
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   발생년         109 non-null    int64         
 1   발생년월일시      109 non-null    datetime64[ns]
 2   주야          109 non-null    object        
 3   요일          109 non-null    object        
 4   사망자수        109 non-null    int64         
 5   부상자수        109 non-null    int64         
 6   중상자수        109 non-null    int64         
 7   경상자수        109 non-null    int64         
 8   부상신고자수      109 non-null    int64         
 9   발생지시도       109 non-null    object        
 10  발생지시군구      109 non-null    object        
 11  사고유형_대분류    109 non-null    object        
 12  사고유형_중분류    109 non-null    object        
 13  사고유형        109 non-null    object        
 14  가해자법규위반     109 non-null    object        
 15  도로형태_대분류    109 non-null    object        
 16  도로형태        109 non-null

교통사고 발생 시간 별/구 별 사망자 현황

In [35]:
fig = px.bar(df1, x = '발생시간', y = '사망자수', color = '발생지시군구', title = '대구 시군구별 교통사고 사망자 현황')
fig.show()

In [36]:
fig = px.bar(df1, x = '사망자수', y = '발생시간', color = '발생지시군구', orientation = 'h', title = '대구 시군구별 교통사고 사망자 현황')
fig.show()

지도 활용하여 교통사고 현황 분석

In [41]:
map = folium.Map(location = [36.351, 127.386])   # 목표로하는 지역의 시청의 좌표를 기준으로 한다.
map

지도에 필요한 정보만 담기 -> 대전의 교통사고 현황을 지도에 표출

In [43]:
df_map = df[df.발생지시도 == '대전']    # 정수형은 사이즈로 할 수 없다.
df_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 3 to 3124
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   발생년         74 non-null     int64         
 1   발생년월일시      74 non-null     datetime64[ns]
 2   주야          74 non-null     object        
 3   요일          74 non-null     object        
 4   사망자수        74 non-null     int64         
 5   부상자수        74 non-null     int64         
 6   중상자수        74 non-null     int64         
 7   경상자수        74 non-null     int64         
 8   부상신고자수      74 non-null     int64         
 9   발생지시도       74 non-null     object        
 10  발생지시군구      74 non-null     object        
 11  사고유형_대분류    74 non-null     object        
 12  사고유형_중분류    74 non-null     object        
 13  사고유형        74 non-null     object        
 14  가해자법규위반     74 non-null     object        
 15  도로형태_대분류    74 non-null     object        
 16  도로형태        74 non-null   

사망자수, 부상자수, 중상자수, 경상자수 전처리 -> 지도에 원을 그리기 위해서

In [44]:
df_map = df_map.astype({'사망자수' : 'float64'})
df_map = df_map.astype({'부상자수' : 'float64'})
df_map = df_map.astype({'중상자수' : 'float64'})
df_map = df_map.astype({'경상자수' : 'float64'})

In [45]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 3 to 3124
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   발생년         74 non-null     int64         
 1   발생년월일시      74 non-null     datetime64[ns]
 2   주야          74 non-null     object        
 3   요일          74 non-null     object        
 4   사망자수        74 non-null     float64       
 5   부상자수        74 non-null     float64       
 6   중상자수        74 non-null     float64       
 7   경상자수        74 non-null     float64       
 8   부상신고자수      74 non-null     int64         
 9   발생지시도       74 non-null     object        
 10  발생지시군구      74 non-null     object        
 11  사고유형_대분류    74 non-null     object        
 12  사고유형_중분류    74 non-null     object        
 13  사고유형        74 non-null     object        
 14  가해자법규위반     74 non-null     object        
 15  도로형태_대분류    74 non-null     object        
 16  도로형태        74 non-null   

사망자수, 경상자수에 따른 CircleMarker 지도에 표출하기

In [47]:
for n in df_map.index:
  cnt = df_map['사망자수'][n] + df_map['경상자수'][n] # 사망자수 + 경상자수 -> CircleMarker의 Size
  folium.CircleMarker([df_map['위도'][n], df_map['경도'][n]],
                      radius = cnt * 10, # 사이즈가 너무 작아서 10을 곱해줌
                      popup = df_map['사고유형'][n],
                      color = '#3186cc', fill_colors = '#3186cc').add_to(map)

map.save('대전교통사고현황.html')

세종시 교통사고 분석

In [51]:
df_saejong = pandas.read_csv('/content/12_20_death.csv', encoding = 'euc-kr')
df_saejong.head(3)

,발생년,발생년월일시,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,사고유형_대분류,사고유형_중분류,사고유형,법규위반,도로형태_대분류,도로형태,당사자종별_1당_대분류,당사자종별_2당_대분류,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
0,2012,2012010101,야간,일,1,1,0,0,0,서울,은평구,차대사람,차도통행중,차도통행중,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,949860,1957179,126.931890,37.612680
1,2012,2012010101,야간,일,1,6,5,0,0,전북,정읍시,차대차,정면충돌,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차,946537,1737695,126.909523,35.633956
2,2012,2012010108,주간,일,1,1,0,0,0,충남,청양군,차량단독,공작물충돌,공작물충돌,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,940016,1832833,126.830281,36.491268


In [52]:
# Subter 문자열 자르기를 위해 데이터타입 변경
df_saejong = df_saejong.astype({'발생년월일시' : 'string'})
df_saejong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37128 entries, 0 to 37127
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   발생년           37128 non-null  int64  
 1   발생년월일시        37128 non-null  string 
 2   주야            37128 non-null  object 
 3   요일            37128 non-null  object 
 4   사망자수          37128 non-null  int64  
 5   사상자수          37128 non-null  int64  
 6   중상자수          37128 non-null  int64  
 7   경상자수          37128 non-null  int64  
 8   부상신고자수        37128 non-null  int64  
 9   발생지시도         37128 non-null  object 
 10  발생지시군구        37128 non-null  object 
 11  사고유형_대분류      37128 non-null  object 
 12  사고유형_중분류      37128 non-null  object 
 13  사고유형          37128 non-null  object 
 14  법규위반          37128 non-null  object 
 15  도로형태_대분류      37128 non-null  object 
 16  도로형태          37128 non-null  object 
 17  당사자종별_1당_대분류  37128 non-null  object 
 18  당사자종별_2당_대분류  37128 non-nu

In [53]:
#2019010100 -> 00 시간으로 분리
df_saejong['발생시간'] = df_saejong['발생년월일시'].str[-2:]
df_saejong['발생시간']

0        01
1        01
2        08
3        10
4        03
         ..
37123    00
37124    15
37125    13
37126    21
37127    07
Name: 발생시간, Length: 37128, dtype: string

In [54]:
# 발생시간 숫자 데이터타입으로 변경
df_saejong = df_saejong.astype({'발생시간' : 'int64'})

In [55]:
# 2019010100 -> 20190101 추출 -> 2019-01-01 (날짜형태 YYYY-MM-DD 형태로 변경)
df_saejong['발생년월일시'] = pandas.to_datetime(df_saejong['발생년월일시'].str[:8],
                                     format = '%Y-%m-%d', errors = 'raise')

In [56]:
df_saejong1 = df_saejong[df_saejong.발생지시도 == '세종']
df_saejong1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 1166 to 36654
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   발생년           152 non-null    int64         
 1   발생년월일시        152 non-null    datetime64[ns]
 2   주야            152 non-null    object        
 3   요일            152 non-null    object        
 4   사망자수          152 non-null    int64         
 5   사상자수          152 non-null    int64         
 6   중상자수          152 non-null    int64         
 7   경상자수          152 non-null    int64         
 8   부상신고자수        152 non-null    int64         
 9   발생지시도         152 non-null    object        
 10  발생지시군구        152 non-null    object        
 11  사고유형_대분류      152 non-null    object        
 12  사고유형_중분류      152 non-null    object        
 13  사고유형          152 non-null    object        
 14  법규위반          152 non-null    object        
 15  도로형태_대분류      152 non-null    objec

In [93]:
# 2013,2013042715,주간,토,1,1,0,0,0,세종,세종,차량단독,전도전복,전도전복,안전운전 의무 불이행...
fig = px.bar(df_saejong1, x = '발생시간', y = '사망자수', color = '발생지시군구', title = '세종 시군구별 교통사고 사망자 현황')
fig.show()

In [59]:
fig = px.bar(df_saejong1, x = '발생시간', y = '사망자수', title = '세종 발생시간별 교통사고 사망자 현황')
fig.show()

교차로 사고가 많지 않을까

In [88]:
fig = px.bar(df_saejong1, x = '발생시간', y = '사망자수', color = '도로형태', title = '세종 발생시간/도로형태별 교통사고 사망자 현황')
fig.show()

In [90]:
fig = px.bar(df_saejong1, x = '도로형태', y = '사망자수', title = '세종 도로형태별 교통사고 사망자 현황')
fig.show()

In [86]:
fig = px.bar(df_saejong1, x = '발생시간', y = '사망자수', color = '사고유형', title = '세종 발생시간/사고유형별 교통사고 사망자 현황')
fig.show()

In [91]:
fig = px.bar(df_saejong1, x = '사고유형', y = '사망자수', title = '세종 사고유형별 교통사고 사망자 현황')
fig.show()

In [68]:
map = folium.Map(location = [36.48, 127.286])   # 목표로하는 지역의 시청의 좌표를 기준으로 한다.
map

In [69]:
df_saejong_map = df[df.발생지시도 == '세종']    # 정수형은 사이즈로 할 수 없다.
df_saejong_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 1166 to 36654
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   발생년           152 non-null    int64  
 1   발생년월일시        152 non-null    int64  
 2   주야            152 non-null    object 
 3   요일            152 non-null    object 
 4   사망자수          152 non-null    int64  
 5   사상자수          152 non-null    int64  
 6   중상자수          152 non-null    int64  
 7   경상자수          152 non-null    int64  
 8   부상신고자수        152 non-null    int64  
 9   발생지시도         152 non-null    object 
 10  발생지시군구        152 non-null    object 
 11  사고유형_대분류      152 non-null    object 
 12  사고유형_중분류      152 non-null    object 
 13  사고유형          152 non-null    object 
 14  법규위반          152 non-null    object 
 15  도로형태_대분류      152 non-null    object 
 16  도로형태          152 non-null    object 
 17  당사자종별_1당_대분류  152 non-null    object 
 18  당사자종별_2당_대분류  152 non-nul

In [83]:
df_saejong_map = df_saejong_map.astype({'사망자수' : 'float64'})
df_saejong_map = df_saejong_map.astype({'사상자수' : 'float64'})  # 데이터셋이 '부상자수'가 아닌 '사상자수'이다.
df_saejong_map = df_saejong_map.astype({'중상자수' : 'float64'})
df_saejong_map = df_saejong_map.astype({'경상자수' : 'float64'})

In [78]:
for n in df_saejong_map.index:
  cnt = df_saejong_map['사망자수'][n] + df_saejong_map['경상자수'][n] # 사망자수 + 경상자수 -> CircleMarker의 Size
  folium.CircleMarker([df_saejong_map['위도'][n], df_saejong_map['경도'][n]],
                      radius = cnt * 10, # 사이즈가 너무 작아서 10을 곱해줌
                      popup = df_saejong_map['사고유형'][n],
                      color = '#3186cc', fill_colors = '#3186cc').add_to(map)

map.save('세종교통사고현황.html')